In [24]:
import numpy as np
from numpy import exp
import scipy
from scipy.special import gamma
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import time
import calendar
from scipy.optimize import minimize
from scipy import integrate

In [54]:
def fx1(a, b, c, t, u):
    if c == 0:
        f = (1+u/b)**(-1*a*t)
    else:    
        f = exp(a * t * gamma(-1*c) * ((b+u)**c - b**c))
    return f

def fx2(a, b, c, theta, t, u):
    e = theta*u-0.5*u**2
    f = fx1(a, b, c, t, e)
    return f

In [57]:
def P_star(S, K, a, b, c, theta, t):
    p = 0.5 + (1/scipy.pi)*integrate.quad(P_star_f, 0, np.inf, args=(S, K, a, b, c, theta, t))[0]
    return p

def P_arg(S, K, a, b, c, theta, t):
    p = 0.5 + (1/np.pi)*integrate.quad(P_arg_f, 0, np.inf, args=(S, K, a, b, c, theta, t))[0]
    return p

In [59]:
def P_star_f(S, K, a, b, c, theta, t, u):
    f = np.real((S/(K*fx2(a, b, c, theta, t, -1)))**(1j*u)*fx2(a, b, c, theta, t, -1j*u)/(1j*u))
    return f

def P_arg_f(S, K, a, b, c, theta, t, u):
    f = np.real((S/(K*fx2(a, b, c, theta, t, -1)))**(1j*u)*fx2(a, b, c, theta, t, -1j*u-1)/(1j*u*fx2(a, b, c, theta, t, -1)))
    return f

In [28]:
def tempered_stable(q, r, S, K, a, b, c, theta, t):
    p = S*exp(-q*t)*P_arg(S, K, a, b, c, theta, t)-K*exp(-r*t)*P_star(S, K, a, b, c, theta, t)
    return p

In [29]:
def loss_function(sigma, BTC_df, S, r, option):
    arpe = 0
    for idx, BTC in BTC_df.iterrows():
        date = BTC['Unnamed: 0'][4:6]
        mon = list(calendar.month_abbr).index(BTC['Unnamed: 0'][6:9].capitalize())
        Year = '20'+BTC['Unnamed: 0'][9:11]
        end_time = pd.datetime.strptime(Year+'-'+str(mon)+'-'+date, '%Y-%m-%d')
        T = end_time - start_time
        T = T.days / 365
        strike = BTC['Strike']
        BTC_df.at[idx, 'FT_price'] = tempered_stable(0, 0, spot_price, strike, T, sigma)
        arpe += abs(BTC['Price(USD)'] - BTC['FT_price'])/BTC['Price(USD)']

    height, width = BTC_df.shape
    arpe /= height
    return arpe

In [33]:
BTC_df = pd.read_csv('BTC.csv')
spot_price = 23129
start_time = pd.datetime.strptime('2023-02-24', '%Y-%m-%d')
s = BTC_df['Price(USD)'].std()
print(s)

4739.533736465831


C:\Users\omen of Hansen\AppData\Local\Temp\ipykernel_3736\851579879.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  start_time = pd.datetime.strptime('2023-02-24', '%Y-%m-%d')


In [69]:
for idx, BTC in BTC_df.iterrows():
    date = BTC['Unnamed: 0'][4:6]
    mon = list(calendar.month_abbr).index(BTC['Unnamed: 0'][6:9].capitalize())
    Year = '20'+BTC['Unnamed: 0'][9:11]
    end_time = pd.datetime.strptime(Year+'-'+str(mon)+'-'+date, '%Y-%m-%d')
    T = end_time - start_time
    T = T.days / 365
    strike = BTC['Strike']
    BTC_df.at[idx, 'tempered_stable'] = tempered_stable(0, 0, spot_price, strike, 1.5, 0.7, 0.5, -0.6, T)

C:\Users\omen of Hansen\AppData\Local\Temp\ipykernel_3736\2081049599.py:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  end_time = pd.datetime.strptime(Year+'-'+str(mon)+'-'+date, '%Y-%m-%d')
C:\Users\omen of Hansen\AppData\Local\Temp\ipykernel_3736\2163325504.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  f = np.real((S/(K*fx2(a, b, c, theta, t, -1)))**(1j*u)*fx2(a, b, c, theta, t, -1j*u-1)/(1j*u*fx2(a, b, c, theta, t, -1)))
C:\Users\omen of Hansen\AppData\Local\Temp\ipykernel_3736\2163325504.py:6: RuntimeWarning: invalid value encountered in power
  f = np.real((S/(K*fx2(a, b, c, theta, t, -1)))**(1j*u)*fx2(a, b, c, theta, t, -1j*u-1)/(1j*u*fx2(a, b, c, theta, t, -1)))
C:\Users\omen of Hansen\AppData\Local\Temp\ipykernel_3736\4128078115.py:6: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being ac

In [70]:
BTC_df.head()

,Unnamed: 0,Mark price,Price(USD),Strike,tempered_stable
0,BTC-29DEC23-5000-C,0.7928,18336.6712,5000,NaN
1,BTC-29DEC23-10000-C,0.5971,13810.3259,10000,NaN
2,BTC-29DEC23-11000-C,0.5593,12936.0497,11000,NaN
3,BTC-29DEC23-12000-C,0.5231,12098.7799,12000,NaN
4,BTC-29DEC23-13000-C,0.4877,11280.0133,13000,NaN
